# Estudo Base

## Setup

In [1]:
import pandas as pd
import re
import os
import joblib
import collections
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy
import nltk
from rouge_score import rouge_scorer
from collections import Counter
import fasttext
import numpy as np
import itertools
import string
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
import math

In [2]:
if os.path.exists('lista_papers'):
    lista_papers = joblib.load('lista_papers')
else:
    list_of_files = os.listdir('scisummnet_release1.1__20190413/top1000_complete')

    list_of_files2= []
    for i in list_of_files:
        nome = 'scisummnet_release1.1__20190413/top1000_complete/' + i + '/' + 'Documents_xml/' +os.listdir(f'scisummnet_release1.1__20190413/top1000_complete/{i}/Documents_xml')[0]
        list_of_files2.append(nome)


    lines = []
    for file in list_of_files2:
        f= open(file,'r',  encoding='utf-8')
        lines.append(f.readlines())
        f.close()

    lista_papers=[]
    for j in lines:
        texto=''
        for i in j:
            texto = texto + str(i)
        lista_papers.append(texto)

        joblib.dump(lista_papers, 'lista_papers')

In [3]:
#Aplicando indices aos papers
indices           = range(0,len(lista_papers))
papers_com_indice = dict(zip(indices, lista_papers))

In [4]:
def preprocessa_intro(corpus: str):
    regIter = re.finditer('(<SECTION.*[Ii][nN][tT][rR][oO][dD][Uu][Cc][Tt][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub("\\t","",txt)
        txt = re.sub('\/','', txt)
        txt = re.sub("\-","",txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub('&amp;quot;', '', txt)
        txt = re.sub('&amp;quot;seed&amp;quot;', '', txt)
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
        lista.append(txt)
    return txt

def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<[Aa][Bb][sS][tT][Rr][aA][cC][tT]>)(.+?)(</[aA][bB][sS][tT][rR][aA][cC][tT]>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub('\/','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub("\-","",txt)
        txt = re.sub("\\t","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub('&amp;quot;', '', txt)
        txt = re.sub('&amp;quot;seed&amp;quot;', '', txt)
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub(r'^(\d)*(\.)?([0-9]{1})?', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
        lista.append(txt)
    return txt


def preprocessa_conclusion(corpus: str):
    regIter1 = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos1 = [ t.group(2) for t in regIter1]
    regIter2 = re.finditer('(<SECTION.*[Aa][Cc][Kk][Nn][Oo][Ww][Ll][Ee][Dd][Gg][Mm][Ee][Nn][Tt][Ss].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos2 = [ t.group(2) for t in regIter2]
    regIter3 = re.finditer('(<SECTION.*[Aa][Cc][Kk][Nn][Oo][Ww][Ll][Ee][Dd][Gg][Ee][Mm][Ee][Nn][Tt][Ss].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos3 = [ t.group(2) for t in regIter3]
    textos = textos1
    for i in textos2:
        if i not in textos:
            textos.append(i)
    for i in textos3:
        if i not in textos:
            textos.append(i)
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub('\/','', txt)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub("\-","",txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\\t","",txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub('&amp;quot;', '', txt)
        txt = re.sub('&amp;quot;seed&amp;quot;', '', txt)
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
    return txt

In [5]:
def get_indices_preprocessamento(x, funcao_preprocessamento):
    """Função para obter os índices dos papers que contém a introdução, abstract ou conclusão
    obs:alterar a o parâmetro funcao_preprocessamento com:
    preprocessa_intro OU preprocessa_abstract OU preprocessa_conclusion"""
    papers_processados=[]
    indices=[]
    for i,j in zip(list(x.keys()),list(x.values())):
        try:
            papers_processados.append(funcao_preprocessamento(j))
            indices.append(i)
        except:
            None
    return indices

In [6]:
indices_papers_com_abstract   = get_indices_preprocessamento(papers_com_indice, preprocessa_abstract)
indices_papers_com_introducao = get_indices_preprocessamento(papers_com_indice, preprocessa_intro)
indices_papers_com_conclusao  = get_indices_preprocessamento(papers_com_indice, preprocessa_conclusion)

In [ ]:
def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<[Aa][Bb][sS][tT][Rr][aA][cC][tT]>)(.+?)(</[aA][bB][sS][tT][rR][aA][cC][tT]>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    return textos

In [24]:
papers_com_indice[13]

'<PAPER>\n  <S sid="0">A Practical Part-Of-Speech Tagger</S>\n  <ABSTRACT>\n    <S sid="1" ssid="1">We present an implementation of a part-of-speech tagger based on a hidden Markov model.</S>\n    <S sid="2" ssid="2">The methodology enables robust and accurate tagging with few resource requirements.</S>\n    <S sid="3" ssid="3">Only a lexicon and some unlabeled training text are required.</S>\n    <S sid="4" ssid="4">Accuracy exceeds 96%.</S>\n    <S sid="5" ssid="5">We describe implementation strategies and optimizations which result in high-speed operation.</S>\n    <S sid="6" ssid="6">Three applications for tagging are described: phrase recognition; word sense disambiguation; and grammatical function assignment.</S>\n    <S sid="7" ssid="7">1 Desiderata Many words are ambiguous in their part of speech.</S>\n    <S sid="8" ssid="8">For example, &amp;quot;tag&amp;quot; can be a noun or a verb.</S>\n    <S sid="9" ssid="9">However, when a word appears in the context of other words, the

In [23]:
indices_papers_com_abstract

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 13,
 14,
 15,
 16,
 18,
 19,
 20,
 21,
 22,
 24,
 31,
 32,
 33,
 35,
 36,
 37,
 39,
 40,
 41,
 43,
 44,
 45,
 47,
 48,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 66,
 71,
 74,
 77,
 80,
 88,
 90,
 91,
 93,
 97,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 222,
 223,
 225,
 226,
 227,
 228,
 229,
 230

In [7]:
print('qtd de papers com abstract', len(indices_papers_com_abstract))
print('qtd de papers com introdução', len(indices_papers_com_introducao))
print('qtd de papers com conclusão', len(indices_papers_com_conclusao))

print('---------------------------------')

print('qtd total de papers', len(lista_papers))

qtd de papers com abstract 924
qtd de papers com introdução 802
qtd de papers com conclusão 831
---------------------------------
qtd total de papers 1009


In [8]:
papers_sem_abstract = dict((key,value) for key, value in papers_com_indice.items() if key  not in  indices_papers_com_abstract)
papers_sem_introducao = dict((key,value) for key, value in papers_com_indice.items() if key  not in  indices_papers_com_introducao)
papers_sem_conclusao = dict((key,value) for key, value in papers_com_indice.items() if key  not in  indices_papers_com_conclusao)

In [9]:
def contador_sessoes(x):
    """input: dicionário de papers
       output: lista contendo o título das sessões"""
    titulos_sessoes=[]
    for i in x.values():
        regIter = re.finditer('(<SECTION(.*?)>)', i, re.DOTALL|re.MULTILINE)
        textos = [ t.group(2) for t in regIter]
        titulos_sessoes.append(textos)

    x=[]
    for i in titulos_sessoes:
        for j in i:
            pattern = r'"(.*?)"'
            matches = re.findall(pattern,str(j))
            pattern2 = r'[0-9]'
            new_string = re.sub(pattern2, '', matches[0])
            x.append(new_string.strip())

    frequency = collections.Counter(x)
    return sorted(frequency.items(), key=lambda x:x[1], reverse=True)

In [10]:
contador_sessoes(papers_sem_abstract)

[('.', 20),
 ('Introduction', 12),
 ('Acknowledgements', 6),
 ('Results', 5),
 ('', 4),
 ('Related Work', 4),
 ('Acknowledgments', 3),
 ('Abstract  Introduction', 3),
 ('Data and Evaluation', 3),
 ('Concluding Remarks', 3),
 ('Conclusion', 2),
 ('Experimental Results', 2),
 ('Conclusions and Future Work', 2),
 ('Conditional Random Fields', 2),
 ('The Joyce System in the Ulysses User Interface', 1),
 ('The Structure of Joyce', 1),
 ('The Text Planner', 1),
 ('The Sentence Planner', 1),
 ('The Linguistic Realizer', 1),
 ('An Example', 1),
 ('ENTER AGENT #&lt;information&gt; OBJECT INCOMPONENT Black Box&gt; LOCATION #&lt;PORT P&gt;',
  1),
 ('Porting the System', 1),
 ('TRANSLATION DELIVERY SYSTEM', 1),
 ('TESTING AND EVALUATING MULTI-ENGINE PERFORMANCE', 1),
 ('CURRENT AND FUTURE WORK', 1),
 ('Input Structure', 1),
 ('System Architecture', 1),
 ('Linguistic Knowledge Bases', 1),
 ('Interfaces', 1),
 ('System Performance', 1),
 ('Status', 1),
 ('. Corpus-based Methods', 1),
 ('Arabic Lang

In [11]:
#Results, Discussion, Concluding remarks, Experiments and Results, Conclustion and Future Research
contador_sessoes(papers_sem_conclusao)

[('.', 176),
 ('Introduction', 69),
 ('Discussion', 27),
 ('', 18),
 ('Experiments', 11),
 ('Related Work', 8),
 ('Results', 8),
 ('Acknowledgment', 7),
 ('Experimental Results', 4),
 ('Evaluation.', 3),
 ('Discussion.', 3),
 ('Summary', 3),
 ('Evaluation', 3),
 ('Acknowledgement', 3),
 ('Support Vector Machines', 3),
 ('Previous Work', 3),
 ('Motivation', 3),
 ('Experiments.', 2),
 ('Summary.', 2),
 ('Related work', 2),
 ('. Results', 2),
 ('Method', 2),
 ('REFERENCES', 2),
 ('INTRODUCTION', 2),
 ('TRANSLATION DELIVERY SYSTEM', 1),
 ('TESTING AND EVALUATING MULTI-ENGINE PERFORMANCE', 1),
 ('CURRENT AND FUTURE WORK', 1),
 ('.. The Amount of Training Data Required', 1),
 ('Grammar Transformation.', 1),
 ('LM Adaptation via Sentence Retrieval.', 1),
 ('Structured Query Models.', 1),
 ('Decision Trees.', 1),
 ('Splitting of the POS Tags.', 1),
 ('Our Tagger.', 1),
 ('Motivation and Theoretical implications.', 1),
 ('A CUG Grammar  Model that Aecomodates Word', 1),
 ('Previous work.', 1),


In [12]:
indice_papers=[]
for idx in list(indices_papers_com_abstract):
    if idx in list(indices_papers_com_introducao):
        if idx in list(indices_papers_com_conclusao):
            indice_papers.append(idx)

In [13]:
print('Quantidade de papers com as três sessões simulataneamente:',len(indice_papers))

Quantidade de papers com as três sessões simulataneamente: 707


In [14]:
indices_faltantes=[]
for i in indices:
    if i not in indice_papers:
        indices_faltantes.append(i)

In [15]:
print('Quantidade de papers sem as três sessões:',len(indices_faltantes))

Quantidade de papers sem as três sessões: 302


In [16]:
indices_treino = indice_papers[0:495]
indices_teste  = indice_papers[495:708]

In [17]:
papers_de_fora = dict((key,value) for key, value in papers_com_indice.items() if key in indices_faltantes)

papers        = dict((key,value) for key, value in papers_com_indice.items() if key in indice_papers)
papers_treino = dict((key,value) for key, value in papers_com_indice.items() if key in indices_treino)
papers_teste  = dict((key,value) for key, value in papers_com_indice.items() if key in indices_teste)

In [18]:
joblib.dump(papers,'papers')

['papers']

# Padrão Ouro

In [19]:
if os.path.exists('lista_padrao_ouro'):
    lista_padrao_ouro = joblib.load('lista_padrao_ouro')
else:
    list_of_files = os.listdir('scisummnet_release1.1__20190413/top1000_complete')

    list_of_files2= []
    for i in list_of_files:
        nome = 'scisummnet_release1.1__20190413/top1000_complete/' + i + '/' + 'summary/' +os.listdir(f'scisummnet_release1.1__20190413/top1000_complete/{i}/summary')[0]
        list_of_files2.append(nome)


    lines = []
    for file in list_of_files2:
        f= open(file,'r', encoding='utf-8' )
        lines.append(f.readlines())
        f.close()

    lista_padrao_ouro=[]
    for j in lines:
        texto=''
        for i in j:
            texto = texto + str(i)
        lista_padrao_ouro.append(texto)

        joblib.dump(lista_padrao_ouro, 'lista_padrao_ouro')

In [20]:
#Aplicando indices aos papers
indices           = range(0,len(lista_papers))
padrao_ouro = dict(zip(indices, lista_padrao_ouro))

In [21]:
selecao_padrao_ouro = dict((key,value) for key, value in padrao_ouro.items() if key in  indice_papers)
selecao_padrao_ouro_treino = dict((key,value) for key, value in padrao_ouro.items() if key in  indices_treino)
selecao_padrao_ouro_teste = dict((key,value) for key, value in padrao_ouro.items() if key in  indices_teste)

In [22]:
joblib.dump(selecao_padrao_ouro,'selecao_padrao_ouro')

['selecao_padrao_ouro']